In [22]:
# Must install psrqpy first: https://github.com/mattpitkin/psrqpy
import psrqpy
from configparser import ConfigParser
from math import sqrt

In [23]:
class Antenna:     
    def __init__(self):
        pass
    
def detectability(SNR,tobs,beta_g,temp,gain,npol,bw):
    #beta_g is beta/gain
    detect = SNR*beta_g*temp/(sqrt(npol*tobs*bw))
    return detect

                                   
# Read antennas parameters file
a_param_location = './antenna_parameters.dat'
a_param = ConfigParser()
a_param.read(a_param_location)

# Antenna 1 parameters
A1 = Antenna()
A1.bw = a_param.getfloat('A1','bandwidth')
A1.nu = a_param.getfloat('A1','nu_central')
A1.gain = a_param.getfloat('A1','gain')
A1.temp = a_param.getfloat('A1','t_sys')
A1.npol = a_param.getfloat('A1','n_pol')
A1.beta_g = a_param.getfloat('A1','beta_g')

# Antenna 2 parameters
A2 = Antenna()
A2.bw = a_param.getfloat('A2','bandwidth')
A2.nu = a_param.getfloat('A2','nu_central')
A2.gain = a_param.getfloat('A2','gain')
A2.temp = a_param.getfloat('A2','t_sys')
A2.npol = a_param.getfloat('A2','n_pol')
A2.beta_g = a_param.getfloat('A2','beta_g')

#Observation parameters
SNR = 10.0
tobs = 240*60.0

In [24]:
#Detectability from condition SNR > 10 -> S1400[Jy]*sqrt((P[s]-W50[s])/W50[s]) > detect
detect1 = detectability(SNR,tobs,A1.beta_g,A1.temp,A1.gain,A1.npol,A1.bw)
detect2 = detectability(SNR,tobs,A2.beta_g,A2.temp,A2.gain,A2.npol,A2.bw)
detect = min(detect1,detect2)

condition1 = 'S1400*sqrt(P0/W50-1.0) > {}'.format(detect)
condition2 = 'decj < -10'
condition_query = condition1 + ' && ' + condition2
#condition_query = psrqpy.QueryATNF.parse_conditions(condition1,condition2) #condition1 + ' && ' + condition2
print(condition_query)

S1400*sqrt(P0/W50-1.0) > 0.006614378277661476 && decj < -10


In [25]:
table = psrqpy.QueryATNF(condition=condition_query)
print(table)

ValueError: "SQRT" is not a supported function